
Run this first to fetch the lat and long


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

file_path = "/content/drive/My Drive/Major Project/rohancolab/dataset_part_25.xlsx"
output_path = "/content/drive/My Drive/Major Project/rohancolab/updated_part_25.xlsx"


GOOGLE_MAPS_API_KEY ="" 


Mounted at /content/drive


In [ ]:




import pandas as pd
import requests
import time
# Load the Excel file
df = pd.read_excel(file_path)
def get_place_details(name, city):
    url = "https://places.googleapis.com/v1/places:searchText"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": GOOGLE_MAPS_API_KEY,
        "X-Goog-FieldMask": "places.displayName,places.rating,places.location,places.userRatingCount"
    }
    payload = {
        "textQuery": f"{name}, {city}",
        "maxResultCount": 1
    }
    response = requests.post(url, json=payload, headers=headers)
    if response.status_code == 200:
        data = response.json()

        if "places" in data and len(data["places"]) > 0:
            place = data["places"][0]  # Taking the first result as the best match
            rating = place.get("rating", None)
            lat = place["location"]["latitude"]
            lng = place["location"]["longitude"]
            rating_tot = data["places"][0].get("userRatingCount", None)
            return rating, lat, lng , rating_tot
    else:
        print(f"Error: {response.status_code}, {response.text}")
    return None, None, None,None
for index, row in df.iterrows():
    if pd.isna(row.get("rating")) or pd.isna(row.get("latitude")) or pd.isna(row.get("longitude")):  # Check if values are missing
        name = row["name"]
        city = row["city"]
        print(f"Searching for {name}, {city} , idx {index}")
        rating, lat, lng, rating_tot = get_place_details(name, city)
        if rating is not None:
            df.at[index, "rating"] = rating  # Update rating column
            df.at[index, "rating_count"] = rating_tot
        if lat is not None and lng is not None:
            df.at[index, "latitude"] = lat  # Update latitude column
            df.at[index, "longitude"] = lng  # Update longitude column
        else:
            print(f"Details not found for {name}, {city}")
        time.sleep(1)
df.to_excel(output_path, index=False)
print("Process completed.")



Searching for Roti Kings, Urapakkam,Chennai , idx 0
Searching for CAFE de Liquid Republic, Urapakkam,Chennai , idx 1
Searching for sri arya bhavan, Urapakkam,Chennai , idx 2
Searching for Kovai Kitchen, Urapakkam,Chennai , idx 3
Searching for AMMA VECHA MEEN KOLAMBU, Urapakkam,Chennai , idx 4
Searching for The Biryani Night, Urapakkam,Chennai , idx 5
Searching for Frankista, Urapakkam,Chennai , idx 6
Searching for AL-NUR HOTEL, Urapakkam,Chennai , idx 7
Searching for Kamali 99 Foods, Urapakkam,Chennai , idx 8
Details not found for Kamali 99 Foods, Urapakkam,Chennai
Searching for ROYAL HYDERABADI BIRYANI, Urapakkam,Chennai , idx 9
Searching for KARISHMA RESTAURANT, Urapakkam,Chennai , idx 10
Searching for BIRYANI POINT, Urapakkam,Chennai , idx 11
Searching for Brownie Island, Urapakkam,Chennai , idx 12
Searching for Shariff Bhai Biryani, Urapakkam,Chennai , idx 13
Searching for V Meet Multicuisine Restaurant, Urapakkam,Chennai , idx 14
Searching for CHENNAI SPICES, Urapakkam,Chennai , i

Run this for main fetch  run only after running the top one

In [ ]:
import requests
import openpyxl
import time
import json
import math
from datetime import datetime, timezone, timedelta
import requests

# Load data from data.json
with open("/content/drive/My Drive/Major Project/rohancolab/data.json", 'r') as json_file:
    place_data = json.load(json_file)

def get_nearby_establishments(lat, lng, radius=500):
    places = []
    url = f"https://places.googleapis.com/v1/places:searchNearby"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": GOOGLE_MAPS_API_KEY,
        "X-Goog-FieldMask": "places.displayName,places.types,places.id,places.priceLevel"
    }

    payload = {
        "maxResultCount": 20,  # Limit results to avoid excessive costs
        "locationRestriction": {
            "circle": {
                "center": {"latitude": lat, "longitude": lng},
                "radius": radius
            }
        }
    }

    while True:
        response = requests.post(url, headers=headers, json=payload)

        if response.status_code == 200:
            result = response.json()
            places.extend(result.get("places", []))  # Add places to list

            # Check if there's a nextPageToken for more results
            next_page_token = result.get("nextPageToken")
            if next_page_token:
                payload["pageToken"] = next_page_token
                time.sleep(1)  # Required delay before requesting the next page
            else:
                break
        else:
            print(f"Error fetching places: {response.status_code} - {response.text}")
            break

    return places

def map_to_broader_category(place_types):
    # Normalize the keys in the JSON file by making them lowercase and replacing spaces with underscores
    normalized_json_keys = {key.lower().replace(" ", "_"): value for key, value in place_data.items()}

    # We'll check if any place types map directly to our normalized JSON categories
    for place_type in place_types:
        broader_category = normalized_json_keys.get(place_type, None)  # Use normalized key lookup
        if broader_category is not None:
            return place_type  # Return the first matched broader category
    return None


# Function to get numeric value for a place category from the normalized JSON data
def get_numeric_value_for_place(place_category):
    # Look up the place category in the normalized JSON data
    normalized_json_keys = {key.lower().replace(" ", "_"): value for key, value in place_data.items()}
    return normalized_json_keys.get(place_category, None)

def calculate_average_traffic(lat, lng, radius=500):
    # Define destinations for traffic checks (some random points within 500m)
    destinations = [
        {"latitude": lat + 0.005, "longitude": lng},
        {"latitude": lat - 0.005, "longitude": lng},
        {"latitude": lat, "longitude": lng + 0.005},
        {"latitude": lat, "longitude": lng - 0.005}
    ]

    url = "https://routes.googleapis.com/directions/v2:computeRoutes"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": GOOGLE_MAPS_API_KEY,
        "X-Goog-FieldMask": "routes.duration,routes.travelAdvisory,routes.legs"
    }

    # Set departure time to 1 minute in the future
    departure_time = (datetime.now(timezone.utc) + timedelta(minutes=1)).isoformat()

    total_time = 0
    count = 0

    for dest in destinations:
        payload = {
            "origin": {"location": {"latLng": {"latitude": lat, "longitude": lng}}},
            "destination": {"location": {"latLng": dest}},
            "travelMode": "DRIVE",
            "routingPreference": "TRAFFIC_AWARE",
            "departureTime": departure_time  # Future timestamp
        }

        time.sleep(1)  # Avoid exceeding rate limits
        response = requests.post(url, headers=headers, json=payload)

        if response.status_code == 200:
            result = response.json()
            if "routes" in result and result["routes"]:
                travel_time = result["routes"][0]["duration"]  # e.g., "300s"
                total_time += int(travel_time.replace("s", ""))  # Convert to seconds
                count += 1
        else:
            print(f"Error: {response.status_code} - {response.text}")

    return (total_time / count) / 60 if count > 0 else None  # Convert to minutes

# Function to calculate the distance between two latitude and longitude points (Haversine formula)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    d_lat = math.radians(lat2 - lat1)
    d_lon = math.radians(lon2 - lon1)
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(d_lon/2) * math.sin(d_lon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    distance = R * c  # Distance in kilometers
    return distance * 1000  # Convert to meters


def find_distance_to_nearest_main_road(lat, lng):
    url = f"https://roads.googleapis.com/v1/snapToRoads?path={lat},{lng}&key={GOOGLE_MAPS_API_KEY}&interpolate=false"
    response = requests.get(url)

    if response.status_code == 200:
        result = response.json()
        if 'snappedPoints' in result:
            nearest_road_location = result['snappedPoints'][0]['location']
            nearest_lat = nearest_road_location['latitude']
            nearest_lng = nearest_road_location['longitude']

            # Use Haversine formula to calculate distance
            distance = calculate_distance(lat, lng, nearest_lat, nearest_lng)
            return distance
        else:
            return None
    else:
        print("Error:", response.status_code, response.text)
        return None

# Function to write the calculated data into the Excel file
def write_data_to_excel(file_path, row_num, avg_pdensity, traffic_rate, visibility, average_price_level):
    # Load the workbook and select the active sheet
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active

    # Write the calculated values into the respective columns
    sheet[f'J{row_num}'] = avg_pdensity  # Assuming AVG_PDENSITY is in column G
    sheet[f'K{row_num}'] = traffic_rate  # Assuming TRAFFIC_RATE is in column H
    sheet[f'L{row_num}'] = visibility  # Assuming VISIBILITY is in column I
    #sheet[f'J{row_num}'] = comp_presence  # Assuming COMPETITION_PRESENCE is in column J
    sheet[f'M{row_num}'] = average_price_level  # Assuming COMPETITION_PRESENCE is in column J

    # Save the workbook to preserve changes
    workbook.save(file_path)

def process_excel_file(file_path):
    # Load the Excel workbook
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active

    # Iterate over the rows in the Excel sheet
    for idx, row in enumerate(sheet.iter_rows(min_row=1356, values_only=True), start=1356):  # Skip header row and start from row 2
        #print(row)

        Name,city,Rating,Rating_Total , cost1 , cuisine , address, Latitude, Longitude, *_ = row
        #Name, Rating, Rating_Total, Latitude, Longitude, *_ = row
        # Ensure latitude and longitude are present
        if Latitude and Longitude:
            print(f"Processing {Name}: Lat {Latitude}, Lng {Longitude} idx {idx}")
            # Get the calculated metrics
            avg_pdensity, traffic_rate, visibility , average_price_level= find_restaurant_details(Latitude, Longitude, Name)
            # Write the calculated data to the Excel file
            write_data_to_excel(file_path, idx, avg_pdensity, traffic_rate, visibility, average_price_level)


def get_place_details(place_id, api_key):
    """
    Use the new Google Places API to get the details of a place by place_id.
    """
    url = f"https://places.googleapis.com/v1/places/{place_id}"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "priceLevel"
    }
    price_level = 0
    time.sleep(1)  # Avoid exceeding rate limits
    response = requests.get(url, headers=headers)
    #print(response.json())
    if response.status_code == 200:
        data = response.json()
        price_level_string = data.get('priceLevel', 'N/A')

        match price_level_string:
            case "PRICE_LEVEL_FREE":
                return 1  # Free
            case "PRICE_LEVEL_INEXPENSIVE":
                return 2  # Inexpensive
            case "PRICE_LEVEL_MODERATE":
                return 3  # Moderate
            case "PRICE_LEVEL_EXPENSIVE":
                return 4  # Expensive
            case "PRICE_LEVEL_VERY_EXPENSIVE":
                return 5  # Very Expensive
            case _:
                return 0  # Default for invalid or missing value


    else:
        print(f"Error fetching place details for place_id {place_id}: {response.status_code} - {response.text}")
        return 'N/A'
def get_place_details1(place_id, api_key,nearby_est):
    for place in nearby_est:
        if place_id == place.get('id'):

            price_level_string = place.get('priceLevel', 'N/A')

            match price_level_string:
                case "PRICE_LEVEL_FREE":
                    return 1  # Free
                case "PRICE_LEVEL_INEXPENSIVE":
                    return 2  # Inexpensive
                case "PRICE_LEVEL_MODERATE":
                    return 3  # Moderate
                case "PRICE_LEVEL_EXPENSIVE":
                    return 4  # Expensive
                case "PRICE_LEVEL_VERY_EXPENSIVE":
                    return 5  # Very Expensive
                case _:
                    return 0  # Default for invalid or missing value

        else:
            print(f"Error fetching place details for place_id {place_id}: {response.status_code} - {response.text}")
            return 'N/A'


def find_restaurant_details(lat, lng, restaurant_type):
    # Get nearby establishments within 1 km
    nearby_establishments = get_nearby_establishments(lat, lng)

    # Initialize a list to store unique place categories
    places = []

    # Initialize a list to store numeric values corresponding to place categories
    numbers = []

    # Initialize a list to store price levels of restaurants
    price_levels = []

    # Iterate through the nearby establishments and check their types against the JSON file
    for place in nearby_establishments:
        place_types = place.get('types', [])  # List of types for the place

        # Try to map the place types to a broader category (only once per place)
        broader_category = map_to_broader_category(place_types)

        if broader_category:
            # Add unique broader category to the places list
            if broader_category not in places:
                places.append(broader_category)

            # Get the numeric value for this broader category from the JSON file
            numeric_value = get_numeric_value_for_place(broader_category)
            #print("Number value ==== ",numeric_value)
            # Only add one numeric value per place
            if numeric_value is not None and broader_category not in numbers:
                numbers.append(int(numeric_value))  # Ensure numeric value is an integer

            # Fetch the price level for restaurant-type places
            if 'restaurant' in place_types:
                place_id = place.get('id')
                #print("plaace id : " , place_id)
                if place_id:
                    price_level = get_place_details(place_id, GOOGLE_MAPS_API_KEY)
                    #print("price",price_levels)
                    # Ensure price_level is numeric (convert if needed) or append 0 if it's invalid
                    try:
                        if price_level is not None and price_level != 'N/A':
                            # Ensure price_level is an integer
                            price_levels.append(int(price_level))
                        else:
                            # Append a default value of 0 if price_level is 'None' or 'N/A'
                            price_levels.append(0)
                    except ValueError:
                        # If price_level is not a valid integer, append 0
                        price_levels.append(0)

    #print(price_levels)
    # Calculate the average population density based on the numbers list
    if numbers:
        Avg_population_density = sum(numbers) / len(numbers)
    else:
        Avg_population_density = 0

    # Get average traffic in the area
    avg_traffic = calculate_average_traffic(lat, lng)

    # Convert traffic time to a severity rating (1-5)
    traffic_severity = avg_traffic
    # Find distance to nearest main road
    distance_to_main_road = find_distance_to_nearest_main_road(lat, lng)

    # Calculate competitor presence for the given coordinates and restaurant type
    #competitor_presence = competitor_presence_for_location(lat, lng, restaurant_type)


    if price_levels:
        average_price_level = sum(price_levels) / len(price_levels)
    else:
        average_price_level = None  # No price levels available

    # Return the calculated metrics
    return Avg_population_density, traffic_severity, distance_to_main_road, average_price_level

# Provide the Excel file path
file_path = output_path
process_excel_file(file_path)


Processing Copper Kitchen: Lat 13.0507151, Lng 80.2037946 idx 1356
Processing 5 Star Biryani: Lat 13.0499929, Lng 80.2134258 idx 1357
Processing HOTEL VEG: Lat 13.0493943, Lng 80.2104155 idx 1358
Processing 24/7 Briyani world: Lat 13.0555762, Lng 80.2110548 idx 1359
Processing Dial A Biryani: Lat 13.0560633, Lng 80.2111946 idx 1360
Processing GELATO ITALIANO: Lat 13.0502573, Lng 80.2095039 idx 1361
Processing Zion Resto: Lat 13.0741709, Lng 80.2085369 idx 1362
Processing Cakehouse: Lat 13.0337146, Lng 80.21597109999999 idx 1363
Processing Madras Military Hotel: Lat 16.3147099, Lng 80.4172474 idx 1364
Processing Meals of India: Lat 13.0508151, Lng 80.209683 idx 1365
Processing Faasos - Wraps & Rolls: Lat 13.0481735, Lng 80.2106888 idx 1366
Processing Firangi Bake: Lat 13.0496121, Lng 80.2113033 idx 1367
Processing Sweet Truth - Cake and Desserts: Lat 13.0496121, Lng 80.2113033 idx 1368
Processing Behrouz Biryani: Lat 13.0481838, Lng 80.2106898 idx 1369
Processing Oven Story Pizza: Lat 1

In [ ]:
# import requests
# import openpyxl
# import time
# import json
# import math


# # Load data from data.json
# with open('/content/drive/My Drive/Major Project/rohancolab/data.json', 'r') as json_file:
#     place_data = json.load(json_file)

# # Function to get nearby establishments within 1 km, including paginated results
# def get_nearby_establishments(lat, lng, radius=500):
#     places = []
#     url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat},{lng}&radius={radius}&key={GOOGLE_MAPS_API_KEY}"

#     while url:
#         response = requests.get(url)
#         if response.status_code == 200:
#             result = response.json()
#             places.extend(result.get('results', []))  # Add new places to the list

#             # Check if there's a next_page_token
#             next_page_token = result.get('next_page_token')
#             if next_page_token:
#                 # Update the URL for the next request with the next_page_token
#                 url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={next_page_token}&key={GOOGLE_MAPS_API_KEY}"
#                 time.sleep(2)  # Required delay before requesting the next page
#             else:
#                 # No more pages to fetch
#                 url = None
#         else:
#             # If the API request fails, stop further requests
#             break

#     return places

# # Function to map Google's place types to a broader category found in the data.json file
# def map_to_broader_category(place_types):
#     # Normalize the keys in the JSON file by making them lowercase and replacing spaces with underscores
#     normalized_json_keys = {key.lower().replace(" ", "_"): value for key, value in place_data.items()}

#     # We'll check if any place types map directly to our normalized JSON categories
#     for place_type in place_types:
#         broader_category = normalized_json_keys.get(place_type, None)  # Use normalized key lookup
#         if broader_category is not None:
#             return place_type  # Return the first matched broader category
#     return None

# # Function to get numeric value for a place category from the normalized JSON data
# def get_numeric_value_for_place(place_category):
#     # Look up the place category in the normalized JSON data
#     normalized_json_keys = {key.lower().replace(" ", "_"): value for key, value in place_data.items()}
#     return normalized_json_keys.get(place_category, None)

# # Function to calculate average traffic using Google Maps Distance Matrix API
# def calculate_average_traffic(lat, lng, radius=500):
#     # Define destinations for traffic checks (some random points within 1km)
#     destinations = [
#         f"{lat + 0.005},{lng}",
#         f"{lat - 0.005},{lng}",
#         f"{lat},{lng + 0.005}",
#         f"{lat},{lng - 0.005}"
#     ]

#     url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={lat},{lng}&destinations={'|'.join(destinations)}&departure_time=now&key={GOOGLE_MAPS_API_KEY}"
#     time.sleep(1)
#     # Make the API request
#     response = requests.get(url)

#     if response.status_code == 200:
#         data = response.json()

#         travel_times = []

#         # Iterate through the response rows and elements
#         for row in data.get('rows', []):
#             for element in row.get('elements', []):
#                 # Try to get 'duration_in_traffic', fallback to 'duration' if not available
#                 if 'duration_in_traffic' in element:
#                     travel_times.append(element['duration_in_traffic']['value'])  # Get traffic time in seconds
#                 elif 'duration' in element:
#                     travel_times.append(element['duration']['value'])  # Fallback to normal travel time

#         if travel_times:
#             avg_traffic_time = sum(travel_times) / len(travel_times)  # Calculate average in seconds
#             return avg_traffic_time / 60  # Convert to minutes
#         else:
#             return None
#     else:
#         print(response)
#         return None

# # Function to convert traffic time to traffic severity (1-5 scale)
# def convert_traffic_time_to_severity(avg_traffic_time):
#     if avg_traffic_time is None:
#         return 0  # No traffic data
#     if avg_traffic_time < 5:
#         return 1  # Low traffic
#     elif avg_traffic_time < 10:
#         return 2  # Moderate traffic
#     elif avg_traffic_time < 15:
#         return 3  # Heavy traffic
#     elif avg_traffic_time < 20:
#         return 4  # Very heavy traffic
#     else:
#         return 5  # Severe traffic

# # Function to find the distance to the nearest main road using Google Maps Roads API
# def find_distance_to_nearest_main_road(lat, lng):
#     url = f"https://roads.googleapis.com/v1/nearestRoads?points={lat},{lng}&key={GOOGLE_MAPS_API_KEY}"
#     time.sleep(1)
#     response = requests.get(url)

#     if response.status_code == 200:
#         result = response.json()
#         if 'snappedPoints' in result:
#             nearest_road_location = result['snappedPoints'][0]['location']
#             nearest_lat = nearest_road_location['latitude']
#             nearest_lng = nearest_road_location['longitude']

#             # Use Haversine formula to calculate distance to the nearest road
#             distance = calculate_distance(lat, lng, nearest_lat, nearest_lng)
#             return distance
#         else:
#             return None
#     else:
#         print(response)
#         return None

# # Function to calculate the distance between two latitude and longitude points (Haversine formula)
# def calculate_distance(lat1, lon1, lat2, lon2):
#     R = 6371  # Earth radius in kilometers
#     d_lat = math.radians(lat2 - lat1)
#     d_lon = math.radians(lon2 - lon1)
#     a = math.sin(d_lat/2) * math.sin(d_lat/2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(d_lon/2) * math.sin(d_lon/2)
#     c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
#     distance = R * c  # Distance in kilometers
#     return distance * 1000  # Convert to meters

# # Function to write the calculated data into the Excel file
# def write_data_to_excel(file_path, row_num, avg_pdensity, traffic_rate, visibility, average_price_level):
#     # Load the workbook and select the active sheet
#     workbook = openpyxl.load_workbook(file_path)
#     sheet = workbook.active

#     # Write the calculated values into the respective columns
#     sheet[f'J{row_num}'] = avg_pdensity  # Assuming AVG_PDENSITY is in column G
#     sheet[f'K{row_num}'] = traffic_rate  # Assuming TRAFFIC_RATE is in column H
#     sheet[f'L{row_num}'] = visibility  # Assuming VISIBILITY is in column I
#     #sheet[f'J{row_num}'] = comp_presence  # Assuming COMPETITION_PRESENCE is in column J
#     sheet[f'M{row_num}'] = average_price_level  # Assuming COMPETITION_PRESENCE is in column J

#     # Save the workbook to preserve changes
#     workbook.save(file_path)

# # Function to process each row of the Excel file and extract the latitude and longitude
# def process_excel_file(file_path):
#     # Load the Excel workbook
#     workbook = openpyxl.load_workbook(file_path)
#     sheet = workbook.active

#     # Iterate over the rows in the Excel sheet
#     for idx, row in enumerate(sheet.iter_rows(min_row=815, values_only=True), start=815):  # Skip header row and start from row 2
#         #print(row)

#         Name,city,Rating,Rating_Total , cost1 , cuisine , address, Latitude, Longitude, *_ = row
#         #Name, Rating, Rating_Total, Latitude, Longitude, *_ = row
#         # Ensure latitude and longitude are present
#         if Latitude and Longitude:
#             print(f"Processing {Name}: Lat {Latitude}, Lng {Longitude}, row {idx}")
#             # Get the calculated metrics
#             avg_pdensity, traffic_rate, visibility , average_price_level= find_restaurant_details(Latitude, Longitude, Name)
#             # Write the calculated data to the Excel file
#             write_data_to_excel(file_path, idx, avg_pdensity, traffic_rate, visibility, average_price_level)
#         print("")

# # Function to find a restaurant's price range using the Google Places API
# def get_place_id(place_name, api_key):
#     """
#     Use Google Places API to find the place_id for a given place name.
#     """
#     url = f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input={place_name}&inputtype=textquery&key={api_key}"
#     time.sleep(1)
#     response = requests.get(url)

#     if response.status_code == 200:
#         data = response.json()
#         if data.get('candidates'):
#             return data['candidates'][0]['place_id']  # Return the first match place_id
#         else:
#             return None
#     else:
#         print(f"Error fetching place_id for {place_name}: {response.status_code}")
#         return None

# def get_place_details(place_id, api_key):
#     """
#     Use Google Places API to get the details of a place by place_id.
#     """
#     url = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=price_level&key={api_key}"
#     time.sleep(1)
#     response = requests.get(url)

#     if response.status_code == 200:
#         data = response.json()
#         if 'result' in data:
#             # Return the price level, or None if not available
#             return data['result'].get('price_level', 'N/A')
#         else:
#             return 'N/A'
#     else:
#         print(f"Error fetching place details for place_id {place_id}: {response.status_code}")
#         return 'N/A'

# # Function to read restaurant names from an Excel file
# def read_place_names_from_excel(file_path):
#     """
#     Read the place names from the Excel file.
#     """
#     place_names = []

#     # Load the workbook and the first sheet
#     workbook = openpyxl.load_workbook(file_path)
#     sheet = workbook.active

#     # Assuming the place names are in the first column (A)
#     for row in sheet.iter_rows(min_row=2, max_col=1, values_only=True):  # Skip the header
#         place_name = row[0]  # Get the place name from column A
#         if place_name:
#             place_names.append(place_name)

#     return place_names

# # Function to update the price range in the Excel file
# def update_excel_with_price_ranges(file_path, price_ranges):
#     """
#     Write the price range values into the Excel file.
#     """
#     workbook = openpyxl.load_workbook(file_path)
#     sheet = workbook.active

#     # Assuming price range should be written to column B (next to place names)
#     for idx, price_range in enumerate(price_ranges, start=2):
#         sheet[f'B{idx}'] = price_range  # Write price range to column B

#     # Save the updated workbook
#     workbook.save(file_path)

# # Main function to process the data
# def process_places_and_update_excel(file_path, api_key):
#     """
#     Main function that reads the Excel file, fetches price range, and updates the Excel.
#     """
#     # Read place names from the Excel file
#     place_names = read_place_names_from_excel(file_path)

#     # List to store price ranges
#     price_ranges = []

#     # Iterate over each place name and fetch its price range
#     for place_name in place_names:
#         print(f"Fetching price range for: {place_name}")

#         # Get place_id using the place name
#         place_id = get_place_id(place_name, api_key)

#         if place_id:
#             # Get the place details (price range) using the place_id
#             price_range = get_place_details(place_id, api_key)
#         else:
#             price_range = 'Not Found'

#         # Append the price range to the list
#         price_ranges.append(price_range)

#     # Update the Excel file with the retrieved price ranges
#     update_excel_with_price_ranges(file_path, price_ranges)
#     print(f"Updated price ranges in {file_path}.")

# # Main function to handle user input and calculate required metrics
# def find_restaurant_details(lat, lng, restaurant_type):
#     # Get nearby establishments within 1 km
#     nearby_establishments = get_nearby_establishments(lat, lng)

#     # Initialize a list to store unique place categories
#     places = []

#     # Initialize a list to store numeric values corresponding to place categories
#     numbers = []

#     # Initialize a list to store price levels of restaurants
#     price_levels = []

#     # Iterate through the nearby establishments and check their types against the JSON file
#     for place in nearby_establishments:
#         place_types = place.get('types', [])  # List of types for the place

#         # Try to map the place types to a broader category (only once per place)
#         broader_category = map_to_broader_category(place_types)

#         if broader_category:
#             # Add unique broader category to the places list
#             if broader_category not in places:
#                 places.append(broader_category)

#             # Get the numeric value for this broader category from the JSON file
#             numeric_value = get_numeric_value_for_place(broader_category)

#             # Only add one numeric value per place
#             if numeric_value is not None and broader_category not in numbers:
#                 numbers.append(int(numeric_value))  # Ensure numeric value is an integer

#             # Fetch the price level for restaurant-type places
#             if 'restaurant' in place_types:
#                 place_id = place.get('place_id')
#                 if place_id:
#                     price_level = get_place_details(place_id, GOOGLE_MAPS_API_KEY)

#                     # Ensure price_level is numeric (convert if needed) or append 0 if it's invalid
#                     try:
#                         if price_level is not None and price_level != 'N/A':
#                             # Ensure price_level is an integer
#                             price_levels.append(int(price_level))
#                         else:
#                             # Append a default value of 0 if price_level is 'None' or 'N/A'
#                             price_levels.append(0)
#                     except ValueError:
#                         # If price_level is not a valid integer, append 0
#                         price_levels.append(0)


#     # Calculate the average population density based on the numbers list
#     if numbers:
#         Avg_population_density = sum(numbers) / len(numbers)
#     else:
#         Avg_population_density = 0

#     # Get average traffic in the area
#     avg_traffic = calculate_average_traffic(lat, lng)

#     # Convert traffic time to a severity rating (1-5)
#     traffic_severity = avg_traffic
#     # Find distance to nearest main road
#     distance_to_main_road = find_distance_to_nearest_main_road(lat, lng)

#     # Calculate competitor presence for the given coordinates and restaurant type
#     #competitor_presence = competitor_presence_for_location(lat, lng, restaurant_type)


#     if price_levels:
#         average_price_level = sum(price_levels) / len(price_levels)
#     else:
#         average_price_level = None  # No price levels available


#     # Return the calculated metrics
#     return Avg_population_density, traffic_severity, distance_to_main_road, average_price_level

# # Provide the Excel file path
# file_path = output_path
# process_excel_file(file_path)


Processing Burger 27: Lat 27.1584943, Lng 78.03995689999999, row 815

Processing Guru Restaurant: Lat 27.2102578, Lng 77.96776349999999, row 816

Processing The Pastry Shop: Lat 27.1581512, Lng 78.0347389, row 817

Processing Hari Chatni: Lat 27.206678, Lng 78.004802, row 818

Processing New Agra Fast Food & Restaurant: Lat 27.1528465, Lng 78.0341129, row 819

Processing PIZZA PASTA DAY: Lat 27.1614253, Lng 77.98344329999999, row 820

Processing Momozilla: Lat 27.1514723, Lng 78.0338271, row 821

Processing Eatco: Lat 27.2075122, Lng 77.9985808, row 822

Processing Faasos - Wraps & Rolls: Lat 26.2716378, Lng 72.997914, row 823

Processing Havmor Havfunn ice cream: Lat 27.2057564, Lng 77.9969472, row 824

Processing Hippo's Krates Cafe: Lat 27.1557027, Lng 78.0356642, row 825

Processing Subway: Lat 27.1601935, Lng 78.0422123, row 826

Processing DIVYA SWEETS AND RESTAURANT: Lat 27.1678585, Lng 78.0422076, row 827

Processing Mahi Food Court: Lat 27.2124756, Lng 77.9855942, row 828

Pro